# BLEND035
- July 30
- Include old models

In [3]:
import pandas as pd
import numpy as np
import os
import random
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error

def group_mean_log_mae(y_true, y_pred, groups, floor=1e-9):
    maes = (y_true-y_pred).abs().groupby(groups).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()

def log_mean_absolute_error(y_true, y_pred):
    return np.log(mean_absolute_error(y_true, y_pred))
%matplotlib inline

In [4]:
BLEND_NUMBER = 'BLEND035'

In [6]:
types = ['1JHC', '2JHH', '1JHN', '2JHN', '2JHC', '3JHC', '3JHH', '3JHN']

def get_sub_oof(t):
    """
    Grab the sub and oof files for a give type. 
    """
    oof = []
    sub = []
    model_ids = []
    for file in sorted(os.listdir(f'../type_results/{t}/')):
        if ('sub' in file) and ('3folds' in file):
            model_id = file[:4] + file.split('MAE_')[1].replace('L','')
            model_ids.append(model_id)
            df = pd.read_parquet(f'../type_results/{t}/{file}')
            if len(sub) == 0:
                sub = df.rename(columns={'scalar_coupling_constant': model_id})
            else:
                sub[model_id] = df['scalar_coupling_constant']
        elif ('oof' in file) and ('3folds' in file):
            model_id = file[:4] + file.split('MAE_')[1].replace('L','')
            #print(model_id)
            df = pd.read_parquet(f'../type_results/{t}/{file}')
            if len(oof) == 0:
                oof = df.rename(columns={'oof_preds': model_id})
            else:
                oof[model_id] = df['oof_preds']
    for file in sorted(os.listdir(f'../type_results/{t}/old/')):
        if ('sub' in file) and ('3folds' in file):
            model_id = file[:4] + file.split('MAE_')[1].replace('L','')
            model_ids.append(model_id)
            df = pd.read_parquet(f'../type_results/{t}/old/{file}')
            if len(sub) == 0:
                sub = df.rename(columns={'scalar_coupling_constant': model_id})
            else:
                sub[model_id] = df['scalar_coupling_constant']
        elif ('oof' in file) and ('3folds' in file):
            model_id = file[:4] + file.split('MAE_')[1].replace('L','')
            #print(model_id)
            df = pd.read_parquet(f'../type_results/{t}/old/{file}')
            if len(oof) == 0:
                oof = df.rename(columns={'oof_preds': model_id})
            else:
                oof[model_id] = df['oof_preds']

    sub = sub.loc[sub['type'] == t]
    oof = oof.loc[oof['type'] == t]
    return sub, oof, model_ids

def get_best_weights(oof, l5, t):
    if len(l5) == 1:
        return [1]
    lmae_best = 0
    for x in tqdm(range(0, 5000)):
        ws = []
        for x in range(0, len(l5)):
            if x == len(l5)-1:
                ws.append(random.randint(1, 10))
            else:
                ws.append(random.randint(0, 10))
        idx = 0
        blend = np.zeros(len(oof[l5[0]]))
        for w in ws:
            blend += w * oof[l5[idx]]
            idx += 1
        blend = blend / np.sum(ws)
        lmae = log_mean_absolute_error(oof['scalar_coupling_constant'],
                            blend)
        if lmae < lmae_best:
            lmae_best = lmae
            bws = ws
    print(f'{t}: ===== Best score {lmae_best:0.5f} - with weights {bws}')
    return bws # Return the best weights

def create_best_blends():
    oofs = []
    subs = []
    for t in types:
        print(f'{t}: Running for type {t}')
        tsub, toof, tmodel_ids = get_sub_oof(t)
        print(f'{t}: Has model names {tmodel_ids}')
        l5 = tmodel_ids[-5:] # Last 5 models
        bws = get_best_weights(toof, l5, t)
        tot_weight = np.sum(bws)
        tsub['scalar_coupling_constant'] = 0
        toof['oof_blend'] = 0
        idx = 0
        for w in bws:
            tsub['scalar_coupling_constant'] += (w * tsub[l5[idx]])
            toof['oof_blend'] += (w * toof[l5[idx]])
            idx += 1
        tsub['scalar_coupling_constant'] /= tot_weight
        toof['oof_blend'] /= tot_weight
        tsub = tsub[['id','molecule_name','type','scalar_coupling_constant']].copy()
        toof = toof[['id','type','scalar_coupling_constant','oof_blend']].copy()
        subs.append(tsub)
        oofs.append(toof)
    final_oof = pd.concat(oofs).sort_values('id').reset_index(drop=True)
    final_sub = pd.concat(subs).sort_values('id').reset_index(drop=True)
    return final_oof, final_sub

final_oof, final_sub = create_best_blends()
print('Done')

1JHC: Running for type 1JHC


  0%|          | 3/5000 [00:00<03:42, 22.42it/s]

1JHC: Has model names ['M048-0.8258', 'M053-0.8639', 'M055-0.8641', 'M047-0.7977', 'M049-0.791']


100%|██████████| 5000/5000 [03:11<00:00, 26.16it/s]


1JHC: ===== Best score -0.99252 - with weights [3, 7, 8, 5, 6]
2JHH: Running for type 2JHH


...

3JHH: ===== Best score -2.31249 - with weights [2, 10, 3, 5, 1]
3JHN: Running for type 3JHN


  0%|          | 6/5000 [00:00<01:23, 59.84it/s]

3JHN: Has model names ['M050-2.383', 'M052-2.39', 'M056-2.4', 'M501-2.416', 'M047-2.356', 'M048-2.321', 'M054-2.277', 'M055-2.363']


100%|██████████| 5000/5000 [01:22<00:00, 60.57it/s]


3JHN: ===== Best score -2.51758 - with weights [9, 7, 4, 1, 5]
Done


# Score

In [7]:
for i, d in final_oof.groupby('type'):
    score = mean_absolute_error(d['scalar_coupling_constant'], d['oof_blend'])
    lscore = np.log(mean_absolute_error(d['scalar_coupling_constant'], d['oof_blend']))
    print(i,'\t {:0.5f} \t {:0.5f}'.format(score, lscore))
glmae = group_mean_log_mae(final_oof['scalar_coupling_constant'], final_oof['oof_blend'], final_oof['type'])
print('\nGroup LMAE')
print('{:0.5f}'.format(glmae))

1JHC 	 0.37064 	 -0.99252
1JHN 	 0.21322 	 -1.54542
2JHC 	 0.14904 	 -1.90355
2JHH 	 0.09829 	 -2.31984
2JHN 	 0.09896 	 -2.31300
3JHC 	 0.16789 	 -1.78445
3JHH 	 0.09901 	 -2.31249
3JHN 	 0.08065 	 -2.51758

Group LMAE
-1.96111


# Validate Sub

In [8]:
sub_good = pd.read_csv('../submissions/BLEND032_sub_-1.98117CV.csv')
final_sub['good_scc'] = sub_good['scalar_coupling_constant']
final_sub[['scalar_coupling_constant','good_scc']].corr()

,scalar_coupling_constant,good_scc
scalar_coupling_constant,1.000000,0.999998
good_scc,0.999998,1.000000


In [9]:
print('MAE vs Good Sub')
for i, d in final_sub.groupby('type'):
    mae_t = mean_absolute_error(d['good_scc'], d['scalar_coupling_constant'])
    corr_t = np.corrcoef(d['good_scc'], d['scalar_coupling_constant'])[1][0]
    print(f'{i} - {mae_t:0.5f} - {corr_t:0.5f}')

MAE vs Good Sub
1JHC - 0.06898 - 0.99998
1JHN - 0.05565 - 0.99997
2JHC - 0.00663 - 1.00000
2JHH - 0.04582 - 0.99981
2JHN - 0.03851 - 0.99985
3JHC - 0.05190 - 0.99957
3JHH - 0.02220 - 0.99995
3JHN - 0.02595 - 0.99942


# Create Submission and OOF Files

In [10]:
import json
import os.path
import re
import ipykernel
import requests

#try:  # Python 3
#    from urllib.parse import urljoin
#except ImportError:  # Python 2
#    from urlparse import urljoin

# Alternative that works for both Python 2 and 3:
from requests.compat import urljoin

try:  # Python 3 (see Edit2 below for why this may not work in Python 2)
    from notebook.notebookapp import list_running_servers
except ImportError:  # Python 2
    import warnings
    from IPython.utils.shimmodule import ShimWarning
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=ShimWarning)
        from IPython.html.notebookapp import list_running_servers


def get_notebook_name():
    """
    Return the full path of the jupyter notebook.
    """
    kernel_id = re.search('kernel-(.*).json',
                          ipykernel.connect.get_connection_file()).group(1)
    servers = list_running_servers()
    for ss in servers:
        response = requests.get(urljoin(ss['url'], 'api/sessions'),
                                params={'token': ss.get('token', '')})
        for nn in json.loads(response.text):
            if nn['kernel']['id'] == kernel_id:
                relative_path = nn['notebook']['path']
                return os.path.join(ss['notebook_dir'], relative_path)

In [11]:
BLEND_NUMBER = get_notebook_name().split('/')[-1].replace('.ipynb','').replace('-','')
print(f'NAME TO SAVE {BLEND_NUMBER}')

NAME TO SAVE BLEND035


In [12]:
# Save the Results
final_sub[['id','scalar_coupling_constant']].to_csv(f'../submissions/{BLEND_NUMBER}_sub_{glmae:0.5f}CV.csv', index=False)
final_oof.to_csv(f'../oof/{BLEND_NUMBER}_oof_{glmae:0.5f}CV.csv', index=False)